# Neptune + Tensorflow/Keras

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Create a `NeptuneCallback()`,
* Log model training metrics to Neptune using `NeptuneCallback()`
* Log additional metadata like hyperaparameters, images, and model weights to Neptune.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client neptune-tensorflow-keras tensorflow == 2.5.0

## Import libraries

In [ ]:
import tensorflow as tf

## Define your model, data loaders, and optimizer

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax),
    ]
)

optimizer = tf.keras.optimizers.SGD(
    lr=0.005,
    momentum=0.4,
)

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

## Quickstart

### Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project="common/tf-keras-integration", api_token="ANONYMOUS")

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next. 

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/tf-keras-integration'` as an anonymous user `'neptuner'` you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#get-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the `project` argument of the `init()` method.

For example:

```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT", 
             api_token="YOUR_API_TOKEN")
```

### Add NeptuneCallback() to model.fit()

In [ ]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

neptune_cbk = NeptuneCallback(run=run, base_namespace="metrics")

model.fit(x_train, y_train, epochs=5, batch_size=64, callbacks=[neptune_cbk])

### Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

### Explore results in the Neptune UI

Go to the run link and explore charts that were created from the metric values you logged. 

## More Options

You can log way more than just metrics to Neptune. 

Let's go over some of those options here. 

### Log hyperparameters

If you want to log hyperparameters of your model training run you just need to pass them to the `base_namespace` of your choice.

In [ ]:
run_2 = neptune.init(project="common/tf-keras-integration", api_token="ANONYMOUS")

params = {"lr": 0.005, "momentum": 0.9, "epochs": 10, "batch_size": 32}

# log hyper-parameters
run_2["hyper-parameters"] = params

optimizer = tf.keras.optimizers.SGD(learning_rate=params["lr"], momentum=params["momentum"])

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

neptune_cbk_2 = NeptuneCallback(run=run_2, base_namespace="metrics")

model.fit(
    x_train,
    y_train,
    epochs=params["epochs"],
    batch_size=params["batch_size"],
    callbacks=[neptune_cbk_2],
)

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next. 

### Log test sample images

In [ ]:
for image in x_test[:100]:
    run_2["test/sample_images"].log(neptune.types.File.as_image(image))

### Log model weights

In [ ]:
import glob

model.save("my_model")

run_2["my_model/saved_model"].upload("my_model/saved_model.pb")

for name in glob.glob("my_model/variables/*"):
    run_2[name].upload(name)

### Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Explore results in the Neptune UI

Go over run details and see all the objects that were logged. 